# Label Shuffle Experiment

In [1]:
import numpy as np
import keras
from joblib import Parallel, delayed
from itertools import product

Load data

In [2]:
(X_train, y_train), (X_test, y_test) = keras.datasets.cifar100.load_data()
data_x = np.concatenate([X_train, X_test])

169009152/169001437 [==============================] - 120s 1us/step


Define hyperparameters
- `num_points_per_task`: The number of points per task
- `shifts`: The number of data ways to split the data into train and test

In [3]:
num_points_per_task = 500

shifts = 2

num_slots = int(5000 // num_points_per_task)
slot_fold = range(int(5000 // num_points_per_task))

Preprocess

In [4]:
# Reshape the data
data_x = data_x.reshape((data_x.shape[0], data_x.shape[1] * data_x.shape[2] * data_x.shape[3]))
data_y = np.concatenate([y_train, y_test])
data_y = data_y[:, 0]

Train model

In [8]:
from functions.label_shuffle_functions import run_parallel_exp

n_trees=[10] # Number of trees in UF

shift_fold = range(1,shifts,1) # Number of shifts
iterable = product(n_trees,shift_fold,slot_fold)

df_list = Parallel(n_jobs=-1,verbose=0)(
    delayed(run_parallel_exp)(
            data_x, data_y, ntree, num_points_per_task, slot=slot, shift=shift
            ) for ntree,shift,slot in iterable
            )

TypeError: __init__() got an unexpected keyword argument 'n_estimators'

Backward transfer efficiency

In [ ]:
from functions.label_shuffle_functions import calc_bte

btes = calc_bte(df_list, num_slots, shifts)

In [ ]:
from functions.label_shuffle_functions import plot_bte

plot_bte(btes)